# Advanced Querying Mongo

**⚠️ IMPORTANT: Limit your print to avoid infinite scrolling. Otherwise your
code will be lost between print lines. If working with lists do:**

```python
list(collection.find(query))[:5] #or a reasonably low number
```

Importing libraries and setting up connection

In [3]:
from pymongo import MongoClient
import pandas as pd
import time
client = MongoClient("localhost:27017")

In [4]:
db = client["ironhack"]

In [5]:
c = db.get_collection("companies")

In [15]:
c.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [10]:
# Your Code
list(c.find({"name":"Babelgum"}, projection = {"name":1, "_id": 0}))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [13]:
# Your Code
list(c.find({"number_of_employees":  {"$gte": 500}},projection = {"_id": 0, "number_of_employees": 1, "name": 1} ).sort("number_of_employees", -1))[:20]

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000},
 {'name': 'Sony', 'number_of_employees': 180500},
 {'name': 'LG', 'number_of_employees': 177000},
 {'name': 'Ford', 'number_of_employees': 171000},
 {'name': 'Boeing', 'number_of_employees': 160000},
 {'name': 'Digital Equipment Corporation', 'number_of_employees': 140000},
 {'name': 'Nokia', 'number_of_employees': 125000},
 {'name': 'MItsubishi Electric', 'number_of_employees': 107000}

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [14]:
# Your Code
list(c.find({"founded_year":  {"$gte":2000,"$lte":2005}},projection = {"_id": 0, "founded_year": 1, "name": 1} ).sort("founded_year", -1))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Jingle Networks', 'founded_year': 2005},
 {'name': 'LifeLock', 'founded_year': 2005},
 {'name': 'Wesabe', 'founded_year': 2005},
 {'name': 'Jangl SMS', 'founded_year': 2005},
 {'name': 'Jajah', 'founded_year': 2005},
 {'name': 'YouTube', 'founded_year': 2005},
 {'name': 'Revision3', 'founded_year': 2005},
 {'name': 'iSkoot', 'founded_year': 2005},
 {'name': 'ClipBlast!', 'founded_year': 2005},
 {'name': 'Zlio', 'founded_year': 2005},
 {'name': 'Dailymotion', 'founded_year': 2005},
 {'name': 'RockYou', 'founded_year': 2005},
 {'name': 'Instructables', 'founded_year': 2005},
 {'name': 'Netvibes', 'founded_year': 2005},
 {'name': 'Swivel', 'founded_year': 2005},
 {'name': 'Slide', 'founded_year': 2005},
 {'name': 'Bebo', 'founded_year': 2005},
 {'name': 'TripHub', 'founded_year': 2005},
 {'name': 'PBworks',

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [18]:
# Your Code
condition_1 = {"ipo.valuation_amount": {"$gt": 100000000}}
condition_2 = {"founded_year":  {"$lte":2010}}
projection = {"_id": 0, "founded_year": 1, "name": 1, "ipo.valuation_amount": 1}
list(c.find({"$and": [condition_1, condition_2]}, projection = projection).sort("ipo.valuation_amount", -1))

[{'name': 'GREE',
  'founded_year': 2004,
  'ipo': {'valuation_amount': 108960000000.0}},
 {'name': 'Facebook',
  'founded_year': 2004,
  'ipo': {'valuation_amount': 104000000000.0}},
 {'name': 'Amazon',
  'founded_year': 1994,
  'ipo': {'valuation_amount': 100000000000.0}},
 {'name': 'Twitter',
  'founded_year': 2006,
  'ipo': {'valuation_amount': 18100000000.0}},
 {'name': 'Groupon',
  'founded_year': 2008,
  'ipo': {'valuation_amount': 12800000000.0}},
 {'name': 'Tencent',
  'founded_year': 1998,
  'ipo': {'valuation_amount': 11000000000.0}},
 {'name': 'Western Digital',
  'founded_year': 1970,
  'ipo': {'valuation_amount': 9430000000.0}},
 {'name': 'LinkedIn',
  'founded_year': 2003,
  'ipo': {'valuation_amount': 9310000000.0}},
 {'name': 'BMC Software',
  'founded_year': 1980,
  'ipo': {'valuation_amount': 6000000000.0}},
 {'name': 'Rackspace',
  'founded_year': 1998,
  'ipo': {'valuation_amount': 5440000000.0}},
 {'name': 'Baidu',
  'founded_year': 1999,
  'ipo': {'valuation_amou

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [19]:
# Your Code
condition_1 = {"number_of_employees":  {"$lt": 100}}
condition_2 = {"founded_year":  {"$lte":2005}}
projection = {"_id": 0, "founded_year": 1, "name": 1, "number_of_employees": 1}
list(c.find({"$and": [condition_1, condition_2]}, projection = projection).sort("number_of_employees", -1))[:10]

[{'name': 'Fulcrum Analytics',
  'number_of_employees': 99,
  'founded_year': 1993},
 {'name': 'PeoplePC', 'number_of_employees': 98, 'founded_year': 1999},
 {'name': 'EffectiveUI', 'number_of_employees': 97, 'founded_year': 2005},
 {'name': 'Smaato', 'number_of_employees': 95, 'founded_year': 2005},
 {'name': 'Volagratis', 'number_of_employees': 95, 'founded_year': 2004},
 {'name': 'Pure Digital Technologies',
  'number_of_employees': 95,
  'founded_year': 2001},
 {'name': 'Clinnovo Research Labs Pvt Ltd',
  'number_of_employees': 95,
  'founded_year': 2004},
 {'name': 'eG Innovations', 'number_of_employees': 95, 'founded_year': 2001},
 {'name': 'Kineto Wireless', 'number_of_employees': 95, 'founded_year': 2001},
 {'name': 'Tableau Software', 'number_of_employees': 92, 'founded_year': 2003}]

### 6. All the companies that don't include the `partners` field.

In [21]:
# Your Code
list(c.find({"partners": { "$exists": "False"}}, projection = {"name": 1, "_id": 0}))[:10]

[{'name': 'Wetpaint'},
 {'name': 'AdventNet'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Facebook'},
 {'name': 'Omnidrive'},
 {'name': 'Postini'},
 {'name': 'Geni'},
 {'name': 'Flektor'},
 {'name': 'Fox Interactive Media'}]

### 7. All the companies that have a null type of value on the `category_code` field.

In [23]:
# Your Code
list(c.find({"category_code" : {"$type": "null"}}, projection = {"name": 1, "_id": 0}))[:10]

[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'},
 {'name': 'Drigg'},
 {'name': 'SpaceTime'},
 {'name': 'Touch Clarity'},
 {'name': 'MMDAYS'},
 {'name': 'Inside Group'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [27]:
# Your Code
list(c.find({"number_of_employees":  {"$gte":100,"$lt":1000}},projection = {"_id": 0, "number_of_employees": 1, "name": 1} ).sort("number_of_employees", -1))[:20]

[{'name': 'Datamonitor', 'number_of_employees': 984},
 {'name': 'Infinera Corporation', 'number_of_employees': 974},
 {'name': 'Box', 'number_of_employees': 950},
 {'name': 'NorthPoint Communications Group', 'number_of_employees': 948},
 {'name': '888 Holdings', 'number_of_employees': 931},
 {'name': 'Forrester Research', 'number_of_employees': 903},
 {'name': 'Workday', 'number_of_employees': 900},
 {'name': 'InMobi', 'number_of_employees': 900},
 {'name': 'Relax Solutions Pvt Ltd', 'number_of_employees': 900},
 {'name': 'SonicWALL', 'number_of_employees': 900},
 {'name': 'Webmetrics', 'number_of_employees': 900},
 {'name': 'Cornerstone OnDemand', 'number_of_employees': 881},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'MySpace', 'number_of_employees': 800},
 {'name': 'ZoomInfo', 'number_of_employees': 800},
 {'name': 'Mozilla', 'number_of_employees': 800},
 {'name': 'Buongiorno', 'number_of_employees': 800},
 {'name': 'Cvent', 'number_of_employees': 800},
 {'name': 'Wayf

### 9. Order all the companies by their IPO price in a descending order.

In [31]:
# Your Code
projection = {"_id": 0, "name": 1, "ipo.valuation_amount": 1}
list(c.find(projection = projection).sort("ipo.valuation_amount", -1))[:15]

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000.0}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000.0}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000.0}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000.0}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000.0}},
 {'name': 'Tencent', 'ipo': {'valuation_amount': 11000000000.0}},
 {'name': 'Western Digital', 'ipo': {'valuation_amount': 9430000000.0}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000.0}},
 {'name': 'BMC Software', 'ipo': {'valuation_amount': 6000000000.0}},
 {'name': 'Rackspace', 'ipo': {'valuation_amount': 5440000000.0}},
 {'name': 'Baidu', 'ipo': {'valuation_amount': 4000000000.0}},
 {'name': 'TripAdvisor', 'ipo': {'valuation_amount': 3273770000.0}},
 {'name': 'HomeAway', 'ipo': {'valuation_amount': 3000000000.0}},
 {'name': 'Zillow', 'ipo': {'valuation_amount': 2550000000.0}},
 {'name': 'Nielsen', 'ipo': {'valuation_amount': 1600000000}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [33]:
# Your Code
projection = {"_id": 0, "name": 1, "number_of_employees": 1}
list(c.find(projection = projection).sort("number_of_employees", -1))[:10]

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [58]:
# Your Code
projection = {"_id": 0, "name": 1, "founded_month": 1}
list(c.find({"founded_month":{"$gte":7,"$lte":12}},projection = projection).sort("founded_month", 1))[:1000]

[{'name': 'Pando Networks', 'founded_month': 7},
 {'name': 'iContact', 'founded_month': 7},
 {'name': 'MeeVee', 'founded_month': 7},
 {'name': 'Yelp', 'founded_month': 7},
 {'name': 'Jaiku', 'founded_month': 7},
 {'name': 'Yapta', 'founded_month': 7},
 {'name': 'Terabitz', 'founded_month': 7},
 {'name': 'GigaOM', 'founded_month': 7},
 {'name': 'Ujogo', 'founded_month': 7},
 {'name': 'Cyworld', 'founded_month': 7},
 {'name': 'Cerado', 'founded_month': 7},
 {'name': 'JumpBox', 'founded_month': 7},
 {'name': 'LongJump', 'founded_month': 7},
 {'name': 'Relationals', 'founded_month': 7},
 {'name': 'Boonex', 'founded_month': 7},
 {'name': 'Rebtel', 'founded_month': 7},
 {'name': 'Buxfer', 'founded_month': 7},
 {'name': 'boo-box', 'founded_month': 7},
 {'name': 'Nirvanix', 'founded_month': 7},
 {'name': 'Metacafe', 'founded_month': 7},
 {'name': 'Sportingo', 'founded_month': 7},
 {'name': 'InviteShare', 'founded_month': 7},
 {'name': 'OpenTable', 'founded_month': 7},
 {'name': 'ChannelAdvisor

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.000

In [41]:
# Your Code
condition_1 = {"founded_year": {"$lt": 2000}}
condition_2 = {"acquisition.price_amount": {"$gt":10000000}}
projection = {"_id": 0, "founded_year": 1, "name": 1, "acquisition.price_amount": 1}
list(c.find({"$and": [condition_1, condition_2]}, projection = projection).sort("acquisition.price_amount", -1))[:10]

[{'name': 'BEA Systems',
  'founded_year': 1995,
  'acquisition': {'price_amount': 8500000000.0}},
 {'name': 'Navteq',
  'founded_year': 1985,
  'acquisition': {'price_amount': 8100000000.0}},
 {'name': 'Sun Microsystems',
  'founded_year': 1982,
  'acquisition': {'price_amount': 7400000000.0}},
 {'name': 'Pixar',
  'founded_year': 1986,
  'acquisition': {'price_amount': 7400000000.0}},
 {'name': 'LSI',
  'founded_year': 1980,
  'acquisition': {'price_amount': 6600000000.0}},
 {'name': 'National Semiconductor',
  'founded_year': 1959,
  'acquisition': {'price_amount': 6500000000.0}},
 {'name': 'aQuantive',
  'founded_year': 1997,
  'acquisition': {'price_amount': 6400000000.0}},
 {'name': 'Siebel Systems',
  'founded_year': 1993,
  'acquisition': {'price_amount': 5850000000.0}},
 {'name': 'Sybase',
  'founded_year': 1984,
  'acquisition': {'price_amount': 5800000000.0}},
 {'name': 'Affiliated Computer Services',
  'founded_year': 1988,
  'acquisition': {'price_amount': 5750000000.0}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [43]:
# Your Code
projection = {"_id": 0, "name": 1, "acquisition": 1}
list(c.find({"acquisition.acquired_year": {"$gt": 2010}}, projection = projection).sort("acquisition.price_amount", -1))[:10]

[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000.0,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000.0,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html',
   'source_description': 'UTC shells out $18.4 billion for Goodrich',
   'acquired_year': 2011,
   'acquired_month': 9,
   'acquired_day': 22,
   'acquiring_company': {'name': 'United Technologies',
    'permalink': 'united-technologies'}}},
 {'name':

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [46]:
# Your Code
projection = {"_id": 0, "name": 1, "founded_year": 1}
list(c.find(projection = projection).sort("founded_year", -1))[:10]

[{'name': 'Fixya', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [48]:
# Your Code
projection = {"_id": 0, "name": 1, "founded_day": 1}
list(c.find({"founded_day": {"$lte":7}},projection = projection).sort("acquisition.price_amount", -1))[:10]

[{'name': 'Netscape', 'founded_day': 4},
 {'name': 'PayPal', 'founded_day': 1},
 {'name': 'Zappos', 'founded_day': 1},
 {'name': 'Alibaba', 'founded_day': 1},
 {'name': 'Postini', 'founded_day': 2},
 {'name': 'Danger', 'founded_day': 1},
 {'name': 'Clearwell Systems', 'founded_day': 6},
 {'name': 'PrimeSense', 'founded_day': 1},
 {'name': 'Amobee', 'founded_day': 1},
 {'name': 'BlueLithium', 'founded_day': 1}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [50]:
# Your Code
projection = {"_id": 0, "name": 1, "category_code": 1}
condition_1 = {"category_code":"web"}
condition_2 = {"number_of_employees":{"$gt":4000}}
list(c.find({"$and": [condition_1,condition_2]},projection = projection).sort("number_of_employees", 1))[:10]

[{'name': 'Expedia', 'category_code': 'web'},
 {'name': 'AOL', 'category_code': 'web'},
 {'name': 'Webkinz', 'category_code': 'web'},
 {'name': 'Rakuten', 'category_code': 'web'},
 {'name': 'Los Angeles Times Media Group', 'category_code': 'web'},
 {'name': 'Groupon', 'category_code': 'web'},
 {'name': 'Yahoo!', 'category_code': 'web'},
 {'name': 'eBay', 'category_code': 'web'},
 {'name': 'Experian', 'category_code': 'web'}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [54]:
# Your Code
projection = {"_id": 0, "name": 1, "acquisition.price_amount": 1, "acquisition.price_currency_code": 1}
condition_1 = {"acquisition.price_currency_code":"EUR"}
condition_2 = {"acquisition.price_amount":{"$gt":10000000}}
list(c.find({"$and": [condition_1,condition_2]},projection = projection).sort("acquisition.price_amount", -1))

[{'name': 'Apertio',
  'acquisition': {'price_amount': 140000000, 'price_currency_code': 'EUR'}},
 {'name': 'Webedia',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'Tuenti Technologies',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'BioMed Central',
  'acquisition': {'price_amount': 43400000, 'price_currency_code': 'EUR'}},
 {'name': 'Greenfield Online',
  'acquisition': {'price_amount': 40000000, 'price_currency_code': 'EUR'}},
 {'name': 'ZYB',
  'acquisition': {'price_amount': 31500000, 'price_currency_code': 'EUR'}},
 {'name': 'Wayfinder',
  'acquisition': {'price_amount': 24000000, 'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [57]:
# Your Code
projection = {"_id": 0, "name": 1, "acquisition": 1}
list(c.find({"acquisition.acquired_month": {"$lte":3}},projection = projection))[:10]

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code

**⚠️ Did you do this?**

```python
list(collection.find(query))[:5] #or a reasonably low number
```